<a href="https://colab.research.google.com/github/NikBaybal/test/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Тестовое задание**


Написать алгоритм, который предложит водителю наиболее подходящие заказы по
пути маршрута.

Входные параметры:
*   Маршрут водителя - координаты точки А и Б
*   Массив заказов, где заказ содержит координаты точки А и Б

**Загрузка библиотек**

In [ ]:
pip install osmnx

In [ ]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim

**Задание входных данных**

Для наглядности и проверки работы алгоритма подбора подходящих заказов задаются определенные адреса в г.Якутске с дальнейшим преобразованием их в географические координаты.

In [ ]:
# задание адресов
routs_address=[['Якутск, ул.Пояркова, 12','Якутск, ул.Ойунского, 27'],
        ['Якутск, ул.Чернышевского, 12','Якутск, ул.Чернышевского, 58'],
        ['Якутск, ул.Лермонтова, 58','Якутск, ул.Ойунского, 8'],
        ['Якутск, ул.Лермонтова, 58','Якутск, ул.Лермонтова, 121']]

In [ ]:
# создание таблицы входных данных
def table(data,columns_name):
  orders_num=['Маршрут водителя']
  for i in range(1,len(data)):
    orders_num.append(f'Заказ №{i}')
  return pd.DataFrame(data, columns=columns_name,index=orders_num)

In [ ]:
table_address=table(routs_address,  ['пункт A','пункт Б'])

In [ ]:
table_address

,пункт A,пункт Б
Маршрут водителя,"Якутск, ул.Пояркова, 12","Якутск, ул.Ойунского, 27"
Заказ №1,"Якутск, ул.Чернышевского, 12","Якутск, ул.Чернышевского, 58"
Заказ №2,"Якутск, ул.Лермонтова, 58","Якутск, ул.Ойунского, 8"
Заказ №3,"Якутск, ул.Лермонтова, 58","Якутск, ул.Лермонтова, 121"


In [ ]:
# функция нахождения георграфических координат по адресу
locator = Nominatim(user_agent = "myapp")
def convert(address):
  location=locator.geocode(address)
  return [location.latitude, location.longitude]

In [ ]:
# представление всех входных адресов в виде географических координат (y-широта, х-долгота)
routs_coord=[]
for i in range(len(routs_address)):
  routs_coord.append(np.concatenate((convert(routs_address[i][0]), convert(routs_address[i][1]))))

In [ ]:
table(routs_coord,  ['A_y','A_x','Б_y','Б_x'])

,A_y,A_x,Б_y,Б_x
Маршрут водителя,62.037927,129.736885,62.020200,129.705274
Заказ №1,62.026819,129.743981,62.011173,129.720808
Заказ №2,62.037553,129.723195,62.025746,129.712052
Заказ №3,62.037553,129.723195,62.010422,129.680913


**Определение кратчайшего расстояния от точки А до точки Б**

Для определения кратчайшего расстояния между двумя точками будет использоваться библиотека OSMnx с уличными сетями из API OpenStreetMap.

In [ ]:
# загрузка карты г.Якутска
ox.config(log_console=True, use_cache=True)
place = 'Yakutsk, Russia'
mode = 'drive'
graph = ox.graph_from_place(place, network_type = mode)

In [ ]:
# функция расчета расстояния от точки А до точки Б
def distance(orig,dest):
  orig_node = ox.nearest_nodes(graph, X=orig[1],Y=orig[0])                                        # нахождение ближайшего узла уличной сети от точки А
  dest_node = ox.nearest_nodes(graph, X=dest[1],Y=dest[0])                                        # нахождение ближайшего узла уличной сети от точки Б
  return (nx.shortest_path_length(graph, orig_node, dest_node, weight='length')/1000).round(1)    # нахождение кратчайшего маршрута А-Б по алгоритму Дейкстры

**Определение подходящих заказов**

Подходящим будет тот заказ, при котором протяженность маршрута Ав-Аз-Бз-Бв будет наименьшей (где Ав-начальная точка водителя, Аз-начальная точка заказа, Бз-конечная точка заказа, Бв-конечная точка водителя ).

In [ ]:
# функция нахождения суммарной протяженности маршрутов Ав-Аз-Бз-Бв
def sum_road(coord):
    sum_distance= [distance(coord[0][:2],coord[0][2:])]
    for i in range(1,len(coord)):
      sum_distance.append(distance(coord[0][:2],coord[i][:2])+distance(coord[i][:2],coord[i][2:])+distance(coord[i][2:],coord[0][2:]))
    return sum_distance


In [ ]:
# вывод таблицы с маршрутами и длиной пути при выполнении каждого заказа
# порядок заказов представлен по приоритету (в порядке убывания длины пути)
road_len=sum_road(routs_coord)
orders_num=['Маршрут водителя']
for i in range(1,len(road_len)):
  orders_num.append(f'Заказ №{i}')
table_address.join(pd.DataFrame(road_len,columns=['Длина пути, км'],index=orders_num)).sort_values('Длина пути, км')

,пункт A,пункт Б,"Длина пути, км"
Маршрут водителя,"Якутск, ул.Пояркова, 12","Якутск, ул.Ойунского, 27",3.1
Заказ №2,"Якутск, ул.Лермонтова, 58","Якутск, ул.Ойунского, 8",4.4
Заказ №1,"Якутск, ул.Чернышевского, 12","Якутск, ул.Чернышевского, 58",5.5
Заказ №3,"Якутск, ул.Лермонтова, 58","Якутск, ул.Лермонтова, 121",6.6
